# Pandas + Seaborn Básico

Notebook guiado para la clase de 3 horas.

Objetivo: cargar, limpiar, agregar y visualizar datos de logs con `pandas` y `seaborn`.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

sns.set_theme(style="whitegrid", context="notebook")
pd.set_option("display.max_columns", 50)

## Setup rápido (ejecutar esto si saltas celdas)

Carga `df` y `clean` para que las celdas avanzadas funcionen sin ejecutar todo el notebook.

## 1) Carga del dataset

Múltiples fuentes: CSV, JSON y creación desde estructuras Python. Parámetros útiles de `read_csv`: `sep`, `encoding`, `parse_dates`, `na_values`.

In [ ]:
# CSV estándar
df = pd.read_csv("data/titanic.csv")

# Ejemplo: crear DataFrame desde lista de dicts (como viene un JSON)
# datos = [{"a": 1, "b": 2}, {"a": 3, "b": 4}]
# df = pd.DataFrame(datos)

print(df.shape)
df.head()

## 2) Inspección profunda

Validar estructura, tipos, duplicados, cardinalidad y patrones de nulos antes de analizar. Detectar problemas temprano evita errores posteriores.

In [ ]:
# Tipos y nulos
print("=== info() ===")
df.info()

# Estadísticas numéricas
print("\n=== describe() ===")
df.describe(include="all").T.head(12)

# Duplicados
print(f"\n=== Duplicados: {df.duplicated().sum()} filas ===")

# Cardinalidad de columnas categóricas
print("\n=== Valores únicos por columna ===")
print(df.select_dtypes(include=["object"]).nunique())

# Patrón de nulos
print("\n=== Nulos por columna ===")
print(df.isna().sum().sort_values(ascending=False))

## 3) Visualización exploratoria en seaborn

Múltiples gráficos para explorar relaciones: countplot, histplot, boxplot por categoría. Usar `order` para ordenar por frecuencia y `hue` para segmentar.

In [ ]:
plt.figure(figsize=(8, 4))
ax = sns.countplot(data=df, x="Pclass")
ax.set_title("Pasajeros por clase")
plt.tight_layout()

## 4) Selección y filtrado

**`loc`**: acceso por etiquetas (nombres de índice y columnas). `iloc`: acceso por posición entera. Siempre usar `loc` para filtros por condición; `iloc` para slices numéricos.

In [ ]:
# loc: por etiquetas (índice y nombres de columna)
# df.loc[filas, columnas]
df.loc[0, "Name"]                    # una celda
df.loc[0:3, ["Name", "Age", "Sex"]]   # filas 0-3 (inclusivas), columnas especificadas

# iloc: por posición entera (como en numpy)
# df.iloc[fila_inicio:fila_fin, col_inicio:col_fin]
df.iloc[0, 3]           # fila 0, columna 3 (Name)
df.iloc[:5, :4]          # primeras 5 filas, primeras 4 columnas
df.iloc[::2, 0]          # filas pares, columna 0

# Diferencia clave: loc incluye el límite final, iloc no (como en Numpy)
print("loc[0:2] incluye fila 2:", df.loc[0:2, "Name"].index.tolist())
print("iloc[0:2] NO incluye fila 2:", df.iloc[0:2, 3].index.tolist())

**Boolean indexing**: combinar condiciones con `&`, `|`, `~`. Siempre usar paréntesis alrededor de cada condición.

In [ ]:
# Ejemplo 1: filtro compuesto con AND
subset_df = df.loc[(df["Pclass"] == 3) & (df["Fare"] < 15), ["Name", "Sex", "Age", "Fare", "Embarked"]]
print(f"Pasajeros en 3a clase con tarifa < 15: {len(subset_df)}")
subset_df.head()

In [ ]:
# Ejemplo 2: filtro con OR
women_or_first_class = df.loc[(df["Sex"] == "female") | (df["Pclass"] == 1)]
print(f"Mujeres O primera clase: {len(women_or_first_class)}")

# Ejemplo 3: filtro con NOT (~)
not_survived = df.loc[~(df["Survived"] == 1)]
print(f"No sobrevivientes: {len(not_survived)}")

# Ejemplo 4: isin para múltiples valores
embarked_sc = df.loc[df["Embarked"].isin(["S", "C"])]
print(f"Embarcados en S o C: {len(embarked_sc)}")

In [ ]:
# Ejemplo 5: between para rangos
adults_young = df.loc[df["Age"].between(18, 35)]
print(f"Adultos jovenes (18-35): {len(adults_young)}")

# Ejemplo 6: query - sintaxis más limpia
queried = df.query("Pclass == 1 and Survived == 1 and Sex == 'female'")
print(f"Mujeres supervivientes de 1a clase: {len(queried)}")

## 5) Limpieza básica de tipos y nulos

Titanic ya viene con nulos reales, ideal para practicar limpieza. Age y Cabin tienen valores faltantes que debemos tratar antes de analizar.

In [ ]:
clean = df.copy()

clean["Age"] = pd.to_numeric(clean["Age"], errors="coerce")
clean["Fare"] = pd.to_numeric(clean["Fare"], errors="coerce")
clean["Embarked"] = clean["Embarked"].fillna(clean["Embarked"].mode()[0])
clean["Age"] = clean["Age"].fillna(clean["Age"].median())

print(clean.isna().sum())

In [ ]:
plt.figure(figsize=(10, 4))
sns.heatmap(df.isna(), cbar=False, cmap="viridis")
plt.title("Mapa de nulos en titanic (amarillo = nulo)")
plt.tight_layout()

## 6) Agregacion y visualizacion

Resumimos por clase y sexo, y vemos supervivencia por edad. Groupby es una de las operaciones más potentes de pandas para análisis exploratorio.

In [ ]:
agg_survival = (
    clean.groupby(["Pclass", "Sex"], as_index=False)
         .agg(passengers=("PassengerId", "count"), survival_rate=("Survived", "mean"))
         .sort_values(["Pclass", "Sex"])
)
agg_survival.head()

In [ ]:
plt.figure(figsize=(8, 4))
sns.barplot(data=agg_survival, x="Pclass", y="survival_rate", hue="Sex")
plt.title("Tasa de supervivencia por clase y sexo")
plt.ylim(0, 1)
plt.tight_layout()

In [ ]:
age_curve = (
    clean.assign(age_bin=(clean["Age"] // 10) * 10)
         .groupby("age_bin", as_index=False)
         .agg(survival_rate=("Survived", "mean"))
         .sort_values("age_bin")
)

plt.figure(figsize=(8, 4))
sns.lineplot(data=age_curve, x="age_bin", y="survival_rate", marker="o")
plt.title("Tasa de supervivencia por rango de edad")
plt.xlabel("Rango de edad (inicio)")
plt.ylabel("Tasa de supervivencia")
plt.ylim(0, 1)
plt.tight_layout()

## 7) Series vs DataFrame

Una Serie es una columna con indice. Un DataFrame es una coleccion de Series.

In [ ]:
# Serie: una columna
ages = clean["Age"]
print(type(ages), ages.shape)

# DataFrame: varias columnas
subset = clean[["Name", "Age", "Sex"]]
print(type(subset), subset.shape)

## 8) Index como ciudadano de primera clase

El indice no es solo decoracion: acelera lookups y facilita joins.

In [ ]:
# Usar PassengerId como indice
indexed = clean.set_index("PassengerId")
indexed.head(3)

# Lookup rápido por indice
indexed.loc[5]

In [ ]:
# MultiIndex: indices jerárquicos
multi_idx = clean.set_index(["Pclass", "Sex"])
multi_idx.head()

# Filtrar por nivel de indice
multi_idx.loc[(3, "male")].head()

## 9) Groupby avanzado: agg, transform, apply

`agg` resume, `transform` mantiene forma original, `apply` es flexible pero lento.

In [ ]:
# agg: resume
clean.groupby("Pclass").agg({"Age": ["mean", "std"], "Fare": "median"})

In [ ]:
# transform: devuelve serie del mismo tamaño que el df original
clean["age_vs_class_mean"] = clean.groupby("Pclass")["Age"].transform("mean")
clean[["PassengerId", "Pclass", "Age", "age_vs_class_mean"]].head()

In [ ]:
# Agregacion con nombres custom (más legible)
clean.groupby("Pclass").agg(
    edad_promedio=("Age", "mean"),
    edad_std=("Age", "std"),
    tarifa_mediana=("Fare", "median"),
    pasajeros=("PassengerId", "count")
).round(2)

In [ ]:
# apply: flexible pero lento - usar solo si no hay alternativa vectorizada
def custom_func(group):
    return group["Age"].max() - group["Age"].min()

clean.groupby("Pclass").apply(custom_func, include_groups=False)

In [ ]:
# Diferencia clave: agg reduce, transform mantiene forma
print("Forma original:", clean.shape)
print("Forma despues de agg:", clean.groupby("Pclass").agg({"Age": "mean"}).shape)

# Con transform obtenemos una columna del tamano original
clean["age_diff_vs_class"] = clean["Age"] - clean.groupby("Pclass")["Age"].transform("mean")
clean[["PassengerId", "Pclass", "Age", "age_vs_class_mean", "age_diff_vs_class"]].head(8)

## 10) Merge / Join

Combinar dataframes por columna o indice. Senior entiende left/right/inner/outer.

In [ ]:
# Crear tabla auxiliar de precios de clase
prices = pd.DataFrame({"Pclass": [1, 2, 3], "base_price": [100, 50, 20]})

# Left join: mantiene todas las filas del df principal
merged = pd.merge(clean.head(10), prices, on="Pclass", how="left")
merged[["PassengerId", "Pclass", "Fare", "base_price"]].head()

In [ ]:
# Join por indice (alternativa a merge)
df1 = clean.set_index("PassengerId")[["Name", "Sex"]].head(5)
df2 = clean.set_index("PassengerId")[["Age", "Fare"]].head(7)

df1.join(df2, how="inner")

## 11) Performance: evitar loops

Nunca iterar fila por fila. Usar operaciones vectorizadas.

In [ ]:
# Comparar diferentes tipos de joins
df_left = clean[["PassengerId", "Name", "Pclass"]].head(5)
df_right = clean[["PassengerId", "Age", "Fare"]].iloc[3:8]

print("Left tiene:", len(df_left), "filas")
print("Right tiene:", len(df_right), "filas")

# Inner: solo coincidencias
inner = pd.merge(df_left, df_right, on="PassengerId", how="inner")
print(f"\nInner join: {len(inner)} filas (solo PassengerId en ambos)")

# Left: mantiene todas de la izquierda
left = pd.merge(df_left, df_right, on="PassengerId", how="left")
print(f"Left join: {len(left)} filas (todas las de df_left)")

# Outer: union de ambas
outer = pd.merge(df_left, df_right, on="PassengerId", how="outer")
print(f"Outer join: {len(outer)} filas (union completa)")

In [ ]:
# MAL: loop
# for i, row in clean.iterrows():
#     clean.at[i, "adult"] = 1 if row["Age"] >= 18 else 0

# BIEN: vectorizacion
clean["adult"] = (clean["Age"] >= 18).astype(int)
clean[["Age", "adult"]].head()

In [ ]:
import numpy as np

# MAL Lento: apply
# clean["fare_category"] = clean["Fare"].apply(lambda x: "high" if x > 30 else "low")

# BIEN Rapido: np.where
clean["fare_category"] = np.where(clean["Fare"] > 30, "high", "low")
clean[["Fare", "fare_category"]].head()

In [ ]:
import time

# Comparacion de rendimiento: loop vs vectorizacion
df_test = clean.copy()

# Opcion 1: loop (LENTO)
start = time.time()
result_loop = []
for idx, row in df_test.head(1000).iterrows():
    result_loop.append(1 if row["Age"] >= 18 else 0)
time_loop = time.time() - start

# Opcion 2: vectorizacion (RAPIDO)
start = time.time()
result_vec = (df_test.head(1000)["Age"] >= 18).astype(int)
time_vec = time.time() - start

print(f"Loop: {time_loop*1000:.2f}ms")
print(f"Vectorizacion: {time_vec*1000:.2f}ms")
print(f"Speedup: {time_loop/time_vec:.0f}x más rápido")

In [ ]:
# np.select para condiciones múltiples (más legible que np.where anidado)
conditions = [
    clean["Fare"] > 50,
    clean["Fare"] > 20,
    clean["Fare"] > 0
]
choices = ["premium", "medium", "low"]
clean["fare_tier"] = np.select(conditions, choices, default="unknown")

clean[["Fare", "fare_tier"]].value_counts("fare_tier").sort_index()

## 12) Optimizar dtypes para reducir memoria

Convertir object a category, float64 a float32 cuando sea posible.

In [ ]:
mem_before = clean.memory_usage(deep=True).sum() / 1024**2

# Convertir columnas categoricas a dtype category
clean["Sex"] = clean["Sex"].astype("category")
clean["Embarked"] = clean["Embarked"].astype("category")

mem_after = clean.memory_usage(deep=True).sum() / 1024**2
print(f"Memoria antes: {mem_before:.2f} MB")
print(f"Memoria despues: {mem_after:.2f} MB")
print(f"Reduccion: {(1 - mem_after/mem_before)*100:.1f}%")

In [ ]:
# Optimizar numericos: float64 -> float32 (cuidado con precision)
clean["Fare_f32"] = clean["Fare"].astype("float32")
print("Memoria Fare float64:", clean["Fare"].memory_usage(deep=True) / 1024, "KB")
print("Memoria Fare float32:", clean["Fare_f32"].memory_usage(deep=True) / 1024, "KB")

# Mostrar todos los dtypes actuales
print("\nDtypes del dataframe:")
clean.dtypes

## 13) Copy vs View (SettingWithCopyWarning)

Evitar modificar vistas implícitas. Usar `.copy()` explícito o `.loc`.

In [ ]:
# MAL TRAMPA: puede generar SettingWithCopyWarning
# subset = clean[clean["Pclass"] == 3]
# subset["new_col"] = 1  # warning!

# BIEN SOLUCION 1: copy explícito
subset = clean[clean["Pclass"] == 3].copy()
subset["new_col"] = 1

# BIEN SOLUCION 2: usar loc desde el inicio
clean.loc[clean["Pclass"] == 3, "new_col"] = 1

## 14) Limpieza avanzada: strings

Metodos `.str` para normalizar texto. Los accessors de string permiten aplicar operaciones de texto de forma vectorizada sobre columnas completas.

In [ ]:
# Extraer informacion de strings con regex
clean["Title"] = clean["Name"].str.extract(r',\s*([^\.]+)\.', expand=False)
print("Titulos unicos encontrados:")
print(clean["Title"].value_counts().head(10))

# Reemplazar y normalizar
clean["Title_clean"] = clean["Title"].str.strip().str.replace("Mlle", "Miss").str.replace("Ms", "Miss")
clean[["Name", "Title", "Title_clean"]].head()

In [ ]:
# Normalizar nombres: lowercase, strip, extrae titulo
clean["name_clean"] = clean["Name"].str.lower().str.strip()
clean["has_mr"] = clean["Name"].str.contains("Mr.", na=False).astype(int)

clean[["Name", "name_clean", "has_mr"]].head()

## 15) Pipelines reproducibles: method chaining

Encadenar operaciones sin variables intermedias. Más legible y reproducible. Este estilo facilita debug y mantiene el código limpio.

In [ ]:
result = (
    clean
    .query("Pclass == 3 and Age > 18")
    .assign(fare_per_year=lambda x: x["Fare"] / x["Age"])
    .groupby("Sex", as_index=False)
    .agg(count=("PassengerId", "count"), avg_fare_per_year=("fare_per_year", "mean"))
    .sort_values("avg_fare_per_year", ascending=False)
)
result

## 18) Pivot table y reshape

Reorganizar datos: pivot_table para resumir, melt para formato largo. Estas transformaciones son fundamentales para preparar datos para visualizacion.

In [ ]:
# Pivot table: tabla resumen con indices y columnas
pivot = clean.pivot_table(
    values="Survived",
    index="Pclass",
    columns="Sex",
    aggfunc="mean"
)
print("Tasa de supervivencia por clase y sexo:")
pivot.round(3)

In [ ]:
# Melt: formato ancho -> largo (util para visualizacion)
sample_wide = clean[["PassengerId", "Age", "Fare"]].head(3)
print("Formato ancho (wide):")
print(sample_wide)

sample_long = sample_wide.melt(id_vars="PassengerId", var_name="metric", value_name="value")
print("\nFormato largo (long) - mejor para seaborn:")
print(sample_long)

## 19) Manejo de fechas y tiempo

Datetime es crucial en análisis real. Pandas tiene herramientas potentes para fechas: parsing, extraccion de componentes, operaciones aritméticas y resampling.

In [ ]:
# Crear data con fechas simuladas
clean_dates = clean.head(100).copy()
clean_dates["boarding_date"] = pd.date_range(start="1912-04-10", periods=100, freq="h")

# Extraer componentes de fecha
clean_dates["year"] = clean_dates["boarding_date"].dt.year
clean_dates["month"] = clean_dates["boarding_date"].dt.month
clean_dates["day"] = clean_dates["boarding_date"].dt.day
clean_dates["hour"] = clean_dates["boarding_date"].dt.hour
clean_dates["day_name"] = clean_dates["boarding_date"].dt.day_name()

clean_dates[["PassengerId", "boarding_date", "year", "month", "day", "hour", "day_name"]].head()

In [ ]:
# Operaciones con fechas
clean_dates["days_since_start"] = (clean_dates["boarding_date"] - clean_dates["boarding_date"].min()).dt.days
clean_dates["in_weekend"] = clean_dates["boarding_date"].dt.dayofweek >= 5

# Resample: agrupar por ventana temporal
time_agg = (
    clean_dates.set_index("boarding_date")
    .resample("6h")["Survived"]
    .agg(["count", "mean"])
)
print("Agregacion por ventanas de 6 horas:")
time_agg.head()

## 20) Validacion y testing de datos

Validar suposiciones y detectar anomalias antes de analizar. Las assertions ayudan a detectar problemás temprano y documentan expectativas sobre los datos.

In [ ]:
# Assertions básicas para validar datos
assert clean.shape[0] > 0, "Dataset vacio!"
assert clean["Age"].min() >= 0, "Edad negativa detectada"
assert clean["Pclass"].isin([1, 2, 3]).all(), "Clase invalida"
print("OK Validaciones básicas pasadas")

# Detectar duplicados
print(f"Filas duplicadas: {clean.duplicated().sum()}")
print(f"PassengerId duplicados: {clean['PassengerId'].duplicated().sum()}")

# Detectar outliers (metodo IQR)
Q1 = clean["Fare"].quantile(0.25)
Q3 = clean["Fare"].quantile(0.75)
IQR = Q3 - Q1
outliers = clean[(clean["Fare"] < Q1 - 1.5*IQR) | (clean["Fare"] > Q3 + 1.5*IQR)]
print(f"Outliers detectados en Fare: {len(outliers)} ({len(outliers)/len(clean)*100:.1f}%)")

## 22) Visualizaciones avanzadas con seaborn

Graficos más complejos para comunicar insights. Boxplots, violinplots, heatmaps y pairplots son herramientas clave para análisis exploratorio.

In [ ]:
# Boxplot: detectar outliers y distribucion
plt.figure(figsize=(10, 5))
sns.boxplot(data=clean, x="Pclass", y="Fare", hue="Survived")
plt.title("Distribucion de tarifa por clase y supervivencia")
plt.tight_layout()
plt.show()

# Violin plot: distribucion + densidad
plt.figure(figsize=(10, 5))
sns.violinplot(data=clean, x="Pclass", y="Age", hue="Survived", split=True)
plt.title("Distribucion de edad por clase (split por supervivencia)")
plt.tight_layout()
plt.show()

In [ ]:
# Heatmap de correlaciones
plt.figure(figsize=(8, 6))
corr_matrix = clean[["Survived", "Pclass", "Age", "Fare"]].corr()
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", center=0, fmt=".2f")
plt.title("Matriz de correlación")
plt.tight_layout()
plt.show()

# Pairplot: relaciones múltiples
sample_for_pair = clean[["Survived", "Pclass", "Age", "Fare"]].dropna().sample(200, random_state=42)
sns.pairplot(sample_for_pair, hue="Survived", diag_kind="kde", corner=True)
plt.suptitle("Pairplot de variables principales", y=1.01)
plt.tight_layout()
plt.show()

## 16) IO serio: parquet y chunking

Parquet es más eficiente que CSV. Chunking para datasets grandes.

In [ ]:
# Guardar y leer en parquet (más rápido y compacto que CSV)
clean.to_parquet("data/titanic.parquet", index=False)
df_parquet = pd.read_parquet("data/titanic.parquet")
df_parquet.head(3)

In [ ]:
# Chunking: procesar datasets grandes por bloques
chunk_results = []
for chunk in pd.read_csv("data/titanic.csv", chunksize=100):
    # Procesar cada chunk
    chunk_agg = chunk.groupby("Pclass")["Survived"].mean()
    chunk_results.append(chunk_agg)

# Combinar resultados
final = pd.concat(chunk_results, axis=1).mean(axis=1)
print("Supervivencia promedio por clase (via chunks):")
print(final)

## 17) Cuando NO usar pandas

Senior sabe cuando pandas NO es la herramienta correcta.

Pandas NO es ideal para:

- **Datasets > 10GB en RAM**: considerar Dask, Polars, PySpark.
- **Joins masivos repetidos**: bases de datos SQL son más eficientes.
- **Procesamiento distribuido**: Spark, Dask distributed.
- **Alta concurrencia**: pandas no es thread-safe.

Alternativas:
- `polars`: más rápido, mejor memoria.
- `dask`: pandas distribuido.
- `pyspark`: big data distribuido.

## 23) Troubleshooting y errores comunes

Soluciones rápidas a problemás frecuentes.

In [ ]:
# Error comun 1: KeyError
try:
    valor = clean["columna_inexistente"]
except KeyError as e:
    print(f"ERROR KeyError: {e}")
    print("Solucion: verificar con clean.columns")

# Error comun 2: ValueError en merge por tipos incompatibles
# Asegurar que las columnas de join tengan el mismo dtype

# Error comun 3: MemoryError
# Solucion: usar chunking, optimizar dtypes, o cambiar a Dask/Polars

# Error comun 4: AttributeError 'Series' object has no attribute 'columns'
# Solucion: verificar si es Series (1D) o DataFrame (2D)
print(f"Type: {type(clean['Age'])} -> usa .shape, no .columns")
print(f"Type: {type(clean[['Age']])} -> tiene .columns")